In [1]:
import numpy as np
import pandas as pd

%matplotlib inline

## Carregando os dados brutos

Os dados brutos possuem 337_500 amostras de sistemas com 1000 pontos por variável de estado (x1, x2)

In [5]:
data = pd.read_feather('../data/raw_data.feather')

Selecionando uma amostra aleatória de 20% (67_500) dos sistemas dos dados brutos. Limpa-se a variável para evitar consumo excessivo de memória

In [4]:
data_sample = data.sample(frac=0.2, random_state=123, ignore_index=True)
data = None
del(data)

## Extraindo as características dos dados brutos

In [5]:
def extract_features(dataframe):
    features = pd.DataFrame()
    features['X1_mean'] = dataframe.iloc[:, :1000].mean(axis=1)
    features['X1_median'] = dataframe.iloc[:, :1000].median(axis=1)
    features['X1_std'] = dataframe.iloc[:, :1000].std(axis=1)
    features['X1_var'] = dataframe.iloc[:, :1000].var(axis=1)
    features['X1_min'] = dataframe.iloc[:, :1000].min(axis=1)
    features['X1_max'] = dataframe.iloc[:, :1000].max(axis=1)
    features['X1_kurtosis'] = dataframe.iloc[:, :1000].kurtosis(axis=1)
    features['X1_skew'] = dataframe.iloc[:, :1000].skew(axis=1)
    features['X1_q1'] = dataframe.iloc[:, :1000].quantile(q=0.25, axis=1)
    features['X1_q2'] = dataframe.iloc[:, :1000].quantile(q=0.5, axis=1)
    features['X1_q3'] = dataframe.iloc[:, :1000].quantile(q=0.75, axis=1)
    
    # Calcula os 45 histogramas para cada linha do DataFrame
    histograms = []
    for i, row in dataframe.iloc[:, :1000].iterrows():
        hist, _ = np.histogram(row, bins=45)
        histograms.append(hist)
        
    histogram_df = pd.DataFrame(histograms, columns=[f'H{n+1}' for n in range(45)])
    df = pd.concat([features, histogram_df], axis=1)
    
    return df

Extraindo as features com a função e adicionando os targets

In [6]:
features = extract_features(data_sample)
features[['y1', 'y2']] = data_sample[['y1', 'y2']]

Salvando os dados em ```.parquet```

In [12]:
features.to_parquet('../data/features_x1.parquet')